In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rheumatoid_Arthritis"
cohort = "GSE224330"

# Input paths
in_trait_dir = "../../input/GEO/Rheumatoid_Arthritis"
in_cohort_dir = "../../input/GEO/Rheumatoid_Arthritis/GSE224330"

# Output paths
out_data_file = "../../output/preprocess/Rheumatoid_Arthritis/GSE224330.csv"
out_gene_data_file = "../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE224330.csv"
out_clinical_data_file = "../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE224330.csv"
json_path = "../../output/preprocess/Rheumatoid_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression of monocytes from rheumatoid arthritis patients treated with bDMARDs and methotrexate."
!Series_summary	"It is well documented that patients affected by rheumatoid arthritis (RA) have distinct susceptibility to the different biologic Disease-Modifying AntiRheumatic Drugs (bDMARDs) available on the market, probably because of the many facets of the disease. Monocytes are deeply involved in the pathogenesis of RA and we therefore evaluated and compared the transcriptomic profile of monocytes isolated from patients on treatment with methotrexate alone or in combination with tocilizumab, anti-TNFalpha or abatacept, and from healthy donors. Differential expression analysis of whole-genome transcriptomics yielded a list of regulated genes suitable for functional annotation enrichment analysis. Specifically, abatacept, tocilizumab and anti-TNFalpha cohorts were separately compared with methotrexate using a rank-product-based statistical a

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Step 1: Determine if gene expression data is available
# From the background information, this dataset contains gene expression profiling
# of monocytes from various patients including RA patients and healthy controls
is_gene_available = True

# Step 2: Determine the availability of trait, age, and gender data
# Looking at the sample characteristics, there is no direct indication of RA status
# The series description mentions RA patients and healthy controls, but this information
# is not directly encoded in the sample characteristics dictionary
trait_row = None  # No direct trait information in sample characteristics

# Age is available in row 1
age_row = 1

# Gender is available in row 2
gender_row = 2

# Define conversion functions
def convert_trait(value):
    # This won't be used since trait_row is None, but we'll define it anyway
    if pd.isna(value):
        return None
    value = value.lower().strip()
    if "rheumatoid arthritis" in value or "ra" in value:
        return 1
    elif "healthy" in value or "control" in value:
        return 0
    else:
        return None

def convert_age(value):
    if pd.isna(value):
        return None
    # Extract the age value which is in format "age: XXy"
    try:
        if ":" in value:
            age_str = value.split(":")[1].strip()
            # Remove the 'y' and convert to integer
            age = int(age_str.replace('y', '').strip())
            return age
        else:
            return None
    except:
        return None

def convert_gender(value):
    if pd.isna(value):
        return None
    # Extract the gender value which is in format "gender: XXX"
    if ":" in value:
        gender = value.split(":")[1].strip().lower()
        if "female" in gender:
            return 0
        elif "male" in gender:
            return 1
        else:
            return None
    else:
        return None

# Step 3: Save metadata
# Determine trait availability - since we don't have a direct trait indicator
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Since trait_row is None, we skip clinical feature extraction


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['A_19_P00315452', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502',
       'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529',
       'A_19_P00315541', 'A_19_P00315543', 'A_19_P00315551', 'A_19_P00315581',
       'A_19_P00315584', 'A_19_P00315593', 'A_19_P00315603', 'A_19_P00315625',
       'A_19_P00315627', 'A_19_P00315631', 'A_19_P00315641', 'A_19_P00315647'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers in the gene expression data

# The identifiers starting with "A_19_P..." appear to be Agilent microarray probe IDs
# rather than standard human gene symbols.
# These are microarray-specific identifiers that would need to be mapped to human gene symbols.
# The format is typical of Agilent platform-specific probe identifiers.

# Human gene symbols would typically look like BRCA1, TP53, IL6, etc.
# The identifiers shown are clearly platform-specific probe IDs that require mapping.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the gene expression data and gene annotation data to identify mapping columns
# The gene expression data uses row IDs similar to A_19_P00315452, A_19_P00315492, etc.
# In the gene annotation data, the 'ID' column appears to contain these same identifiers.
# The 'GENE_SYMBOL' column contains the human gene symbols we want to map to.

# 2. Extract the mapping data from the gene annotation dataframe
# Create a mapping dataframe with the probe IDs and corresponding gene symbols
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This function handles the many-to-many relationships between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Preview the first few rows of the mapped gene data
print("First few rows of the mapped gene expression data:")
print(gene_data.head())
print(f"Shape of gene expression data: {gene_data.shape}")


First few rows of the mapped gene expression data:
          GSM7019507  GSM7019508  GSM7019509  GSM7019510  GSM7019511  \
Gene                                                                   
A1BG        9.809589   10.329705    9.745500   10.834169    9.798319   
A1BG-AS1    7.160811    6.989542    7.031746    7.916862    7.645764   
A1CF        6.110426    6.359480    6.127004    5.932559    6.170733   
A1CF-2      6.161767    5.948398    5.868721    5.825201    5.901578   
A1CF-3      6.081881    6.855441    6.619097    6.117701    6.343309   

          GSM7019512  GSM7019513  GSM7019514  GSM7019515  GSM7019516  ...  \
Gene                                                                  ...   
A1BG        9.578622    9.730334    9.686282   10.763011   10.080284  ...   
A1BG-AS1    6.965617    7.302826    7.289203    7.590010    6.989714  ...   
A1CF        6.177787    6.029664    6.195742    6.116004    6.805011  ...   
A1CF-2      6.015555    6.008692    5.965431    6.086006   

### Step 7: Data Normalization and Linking

In [8]:
# Since trait_row is None (meaning we don't have direct trait information), 
# we can't extract clinical features or properly link the data as originally planned

# We skip clinical feature extraction and go directly to gene data normalization and validation
print("No trait information available in the sample characteristics, skipping clinical feature extraction")

# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since we don't have trait information, we can't create a linked dataset for association analysis
# We need to provide a valid DataFrame and is_biased value for final validation
# Since there's no trait information, we can't assess bias, so we'll set it to False
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False,
    is_biased=False,  # Cannot be None for final validation
    df=normalized_gene_data,  # Provide actual data instead of empty DataFrame
    note="Dataset contains gene expression data but lacks rheumatoid arthritis trait information in the sample characteristics"
)

print("Dataset contains gene expression data but lacks the trait information needed for association analysis")

No trait information available in the sample characteristics, skipping clinical feature extraction
Normalized gene data shape: (20778, 31)


Normalized gene data saved to ../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE224330.csv
Dataset contains gene expression data but lacks the trait information needed for association analysis
